In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-606b2b38-89a5-4f60-8340-a86d063a1507"

In [3]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [4]:
sumids

['8GFXEKD5tgiOPWIPJ2fwG2gNS4igoK4WmwjEEIhjyzQIPVA',
 'AxJCINgn7oZ1v8X9sYZLGTF8f5FpodDRQslkC1rNaVfW-kY',
 '46MhHF_vw8BAsjiv7qIxvdsVu3OfSQ8z-rrS5S3mu2PuNKs',
 'ZgxS2isdqId7b1MuGT4jzB2MiOESPnUedawZkioaYoG38Vw',
 'zzLQAY_q0W76m9sgR4j8rSzQnHdvg9MA0zjw5c410BK6kg',
 'tkYow6ihaHwy3lw7LmUWQF_gWetyWqHOojWSLUudCQr7wfI',
 'm2k9SDoQxoibU6n8Z2pPbC_76kpryQIOrMarJYU-8ad03XA',
 'CpruRt4rRYJ4oxc97vzS_WeqOhvWL_UtQKfzzZDBs0qG4R0',
 'UY2jwuoQswBdkrwXtPE51PHUPwMBlgXAKw3HaPEDagildbQ',
 'X4kmMuxiUsoQfZXm4D6639iQYzNna8GRhoMY0cCvCSbSuqA',
 'gj1qKB885vhIhA0sbIqvlphJ_vQ04Z0ctrV8ioF5LlD5Xko',
 'ERuN9H6ijh_1REFL9IVRlubxuDKN2hyxG3F89w26blqtlhg',
 'SezLwzBKe6HElFDUJHRJMAqI2UsiTDDdyu1WZgmCpzULPfo',
 'lQ8oPPe0UMY3x3crKZ4Vsqqo_A5QRzJIz-q9Bi-A-m_2XKQ',
 'm-rM7N7U1XJoi4bby8Ac9ZEpnlJpL6P0Bvc4FOPRyNX4Lss',
 'yVMpMRee76MrH8KP6j635nnGdcLZVh5jE7bBrw7A-Rkl4Pc',
 'O6Cn2X2nszBpL22wSJ7Kg_-rUu9H4vpT0SOJR0Khc2IIDEg',
 '4_oa0iEtfZF9qMQ6FbbXfXgWfWOItxHCrzg_Um53oqNwqg',
 '5EtxSzngZUxLnW4nwFAYOr-XEbkMBFUYXCPsJVTR7byNpKQ',
 'zGLFmUy3PUf-

In [5]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [6]:
accids[0]

'AMaD_L6Bt2ZzBFfCzT_cfZYy2ARC9tckhMEldkk61UOQtxNp_etBXRqA'

In [7]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
    
    i=0
    while matlreq.status_code != 200:
        matlreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            assert 0 == 1
    
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [8]:
matlreq.status_code

200

In [9]:
len(gamids)

20500

In [28]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = list(set(gamids))

print(len(gamids))

9191
9191


In [11]:
with open('kr_chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [12]:
challgames =set([int(e) for e in challgames])

In [13]:
len(challgames)

9844

In [14]:
len(gamids)

12966

In [15]:
len(gamids - (challgames & gamids))

9191

In [16]:
gamids = gamids - (challgames & gamids)

In [17]:
with open('gm_gam_id_list.txt', 'w') as f:
    for item in gamids:
        f.write("%s\n" % item)

In [29]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids[8892:]:
    gamreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://kr.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            assert 1 == 0
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [30]:
gamreq.status_code

200

In [31]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(299, 1550, 1210)

In [32]:
# create df from matches

df = pd.DataFrame(matchlist)

In [33]:
#df1 = pd.DataFrame(matchlist)

In [34]:
df = df1.append(df)

In [35]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end,participants_9_timeline_creepsPerMinDeltas_30-end,participants_9_timeline_xpPerMinDeltas_30-end,participants_9_timeline_goldPerMinDeltas_30-end,participants_9_timeline_csDiffPerMinDeltas_30-end,participants_9_timeline_xpDiffPerMinDeltas_30-end,participants_9_timeline_damageTakenPerMinDeltas_30-end,participants_9_timeline_damageTakenDiffPerMinDeltas_30-end
0,4425351168,KR,1591187635836,999,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4518445060,KR,1595171405585,967,430,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4540137476,KR,1596037206029,754,420,11,13,10.15.330.344,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4481351685,KR,1593583365162,1802,440,11,13,10.13.326.4870,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4422369287,KR,1591035247840,1890,420,11,13,10.11.322.2991,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,4537286620,KR,1595941850572,1157,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,4521033694,KR,1595313583603,1556,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,4526079969,KR,1595520262043,1238,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,4541677552,KR,1596114756125,913,420,11,13,10.15.330.344,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
gmgames = list(df['gameId'])

In [ ]:
len(set(list(df['gameId'])))

In [36]:
df.to_csv('krgm.zip', index=False,
          compression = dict(method='zip',archive_name='krgm.csv') )